In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("12882_2021_2402_MOESM8_ESM.csv")

In [ ]:
import pandas as pd
from kfre import RiskPredictor

# Sample data
## Note:
data = pd.DataFrame(
    {
        "Age": [65, 70],
        "Gender": ["male", "female"],
        "eGFR": [20, 50],
        "Urinary_Albumin_Creatinine_Ratio": [30, 35],  # in mg/g
        "Region": ["North America", "Other"],
        "Diabetes": [1, 0],
        "Hypertension": [1, 1],
    }
)


# Column mapping
columns = {
    "age": "Age",
    "sex": "Gender",
    "eGFR": "eGFR",
    "uACR": "Urinary_Albumin_Creatinine_Ratio",
    "region": "Region",
    "dm": "Diabetes",
    "htn": "Hypertension",
}

# Initialize the predictor
predictor = RiskPredictor(data, columns)

# Predict CKD risk for 2 years using basic 4-variable model
data["4var_2_Year_Risk"] = predictor.predict_kfre(
    years=2, is_north_american=True, num_vars=4
)

# Predict CKD risk for 5 years using basic 4-variable model
data["4var_5_Year_Risk"] = predictor.predict_kfre(
    years=5, is_north_american=True, num_vars=4
)
data

In [ ]:
import pandas as pd
from kfre import RiskPredictor # type: ignore

# Expanded sample data
## Note: Ensure that the units for each laboratory value are consistent with your model's requirements.
data = pd.DataFrame(
    {
        "Age": [65, 70],
        "Gender": ["male", "female"],
        "eGFR": [20, 50],  # eGFR in mL/min/1.73 m^2
        "Urinary_Albumin_Creatinine_Ratio": [30, 35],  # uACR in mg/g
        "Diabetes": [1, 0],  # 1 for diabetes, 0 for no diabetes
        "Hypertension": [1, 1],  # 1 for hypertension, 0 for no hypertension
        "Serum_Albumin": [3.5, 4.0],  # Serum albumin in g/dL
        "Serum_Phosphorus": [3.5, 4.1],  # Serum phosphorus in mg/dL
        "Serum_Bicarbonate": [24, 22],  # Serum bicarbonate in mEq/L
        "Serum_Calcium": [9.5, 9.7],  # Serum calcium in mg/dL
        "Region": ["North America", "Other"],
    }
)

# Column mapping, including additional columns for the 6-variable and 8-variable models
columns = {
    "age": "Age",
    "sex": "Gender",
    "eGFR": "eGFR",
    "uACR": "Urinary_Albumin_Creatinine_Ratio",
    "dm": "Diabetes",
    "htn": "Hypertension",
    "albumin": "Serum_Albumin",
    "phosphorous": "Serum_Phosphorus",
    "bicarbonate": "Serum_Bicarbonate",
    "calcium": "Serum_Calcium",
}

# Initialize the predictor
predictor = RiskPredictor(data, columns)

# Predict CKD risk for 2 years using the 4-variable model
data["4var_2_Year_Risk"] = predictor.predict_kfre(
    years=2, is_north_american=True, num_vars=4
)

# Predict CKD risk for 5 years using the 6-variable model
data["6var_5_Year_Risk"] = predictor.predict_kfre(
    years=5, is_north_american=True, num_vars=6
)

# Predict CKD risk for 5 years using the 8-variable model
data["8var_5_Year_Risk"] = predictor.predict_kfre(
    years=5, is_north_american=True, num_vars=8
)

data

In [ ]:
from kfre import uPCR_to_uACR

In [ ]:
conversion_columns = {
    "uPCR_mmol": "uPCR",
    "calcium_mmol": "Calcium (mmol/L)",
    "phosphate_mmol": "Phosphate (mmol/L)",
    "albumin_g_per_l": "Albumin (g/l)",
}

predictor = RiskPredictor(data=df, columns=conversion_columns)
predictor.perform_conversions()

In [ ]:
df["uACR"] = df.apply(
    lambda row: uPCR_to_uACR(
        row=row,
        sex_col="SEX",
        diabetes_col="Diabetes (1=yes; 0=no)",
        hypertension_col="Hypertension (1=yes; 0=no)",
        uPCR_col="uPCR (mg/g)",
        female_str="Female",
    ),
    axis=1,
)

In [ ]:
columns = {
    "age": "Age",
    "sex": "SEX",
    "eGFR": "eGFR-EPI",
    "uACR": "uACR",
    "region": "Region",
    "dm": "Diabetes (1=yes; 0=no)",
    "htn": "Hypertension (1=yes; 0=no)",
    "albumin": "Albumin (g/dL)",
    "phosphorous": "Phosphate (mg/dL)",
    "bicarbonate": "Bicarbonate (mmol/L)",
    "calcium": "Calcium (mg/dL)",
}

predictor = RiskPredictor(data=df, columns=columns)

# Predict risk for different combinations of years and variables
pred_4var_2year = predictor.predict_kfre(years=2, is_north_american=False)
pred_4var_5year = predictor.predict_kfre(years=5, is_north_american=False)
pred_6var_2year = predictor.predict_kfre(
    years=2, is_north_american=False, use_extra_vars=True, num_vars=6
)
pred_6var_5year = predictor.predict_kfre(
    years=5, is_north_american=False, use_extra_vars=True, num_vars=6
)

pred_8var_2year = predictor.predict_kfre(
    years=2, is_north_american=False, use_extra_vars=True, num_vars=8
)
pred_8var_5year = predictor.predict_kfre(
    years=5, is_north_american=False, use_extra_vars=True, num_vars=8
)

# inner join the different combinations of years and variables to
# kfre_inclusion dataframe
df_kfre = df.assign(
    pred_4var_2year=pred_4var_2year,
    pred_4var_5year=pred_4var_5year,
    pred_6var_2year=pred_6var_2year,
    pred_6var_5year=pred_6var_5year,
    pred_8var_2year=pred_8var_2year,
    pred_8var_5year=pred_8var_5year,
)

In [ ]:
df_kfre